# Projet SER

In [ ]:
!pip install numpy pandas scikit-learn
!pip install soundfile
!pip install librosa
!pip install kaggle

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, confusion_matrix, classification_report, precision_score, recall_score
import librosa
from scipy.signal import find_peaks
import soundfile as sf
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'seaborn'

## Data Collection

### From local path

In [ ]:
"""
def collect_data_ravdess(audio_path):
    data = pd.DataFrame()

    for file in os.listdir(audio_path):
        if file.endswith(".wav"):
            filepath = os.path.join(audio_path, file)
            label = file.split("-")[2]  # Extracting emotion label from filename

            # Append the filepath and label to the DataFrame
            data = data.append({"filepath": filepath, "label": label}, ignore_index=True)

    return data

"""

### From kaggle

In [2]:
import json
import kaggle

In [3]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

mkdir: /Users/siakesophie/.kaggle: File exists


Après la création de `~/.kaggle` dans votre ordinateur, il faut ouvrir le fichier créer suite à cette commande `!touch ~/.kaggle/kaggle.json` et y mettre vos identifiant kaggle. Cela est nécessaire pour avoir accès au datasets de kaggle.



In [4]:
!echo '{"username":"siakesophie","key":"a14396222d9d4576cce4412a57731f71"}' > ~/.kaggle/kaggle.json


In [6]:
api_token = {"username":"siakesophie","key":"a14396222d9d4576cce4412a57731f71"}

with open('/Users/siakesophie/.kaggle/kaggle.json', 'w') as file:
        json.dump(api_token, file)

In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
# list datasets on kaggle with the name Ravdess

!kaggle datasets list -s RAVDESS

ref                                           title                                  size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------  ------------------------------------  -----  -------------------  -------------  ---------  ---------------  
uwrfkaggler/ravdess-emotional-speech-audio    RAVDESS Emotional speech audio        429MB  2019-01-19 18:28:31          38697        474  0.875            
uwrfkaggler/ravdess-emotional-song-audio      RAVDESS Emotional song audio          456MB  2019-01-19 21:01:49           1939         39  0.875            
dmitrybabko/speech-emotion-recognition-en     Speech Emotion Recognition (en)       987MB  2021-01-25 12:59:50           8542         93  0.875            
kartik2khandelwal/speech-emotion-dataset      RAVDESS as .csv                       763KB  2021-09-15 13:12:51            693         17  0.5882353        
uwrfkaggler/ravdess-facial-landmark-tracking  RAVDESS Facial Lan

In [13]:
# Téléchargement des données sous format csv

kaggle.api.dataset_download_files('kartik2khandelwal/speech-emotion-dataset', path='.', unzip=True)


## Data Processing
- Audio Normalization
- Silence Removal

In [ ]:
def normalize(X):
    max_X = np.max(np.abs(X))
    Y = X / max_X

    return Y

In [ ]:
def remove_silence(X, factor):
    max_X = np.max(np.abs(X))
    decision_threshold = max_X / factor

    # Find the indices of samples above the decision threshold
    indices_useful_X, _ = find_peaks(np.abs(X), height=decision_threshold)

    # Extract the useful samples
    Y = X[indices_useful_X[0]:indices_useful_X[-1] + 1]

    return Y


In [ ]:
def normalise_remove_silence(input_filename, output_directory, factor):
    # Read the input audio file
    x, fs = librosa.load(input_filename, sr=None)

    # Normalize the audio
    y = normalize(x)

    # Remove silence
    z = remove_silence(y, factor)

    # Extract the filename and extension
    filename, extension = os.path.splitext(os.path.basename(input_filename))

    # Create the output filename in the specified directory
    output_filename = os.path.join(output_directory, f"{filename}_N_RS{factor}{extension}")

    # Write the processed audio to the output file
    sf.write(output_filename, z, fs)

## Audio Feature Extraction


In [ ]:
from scipy.stats import mode

def global_feature_computation(feature_matrix, computations):
    computed_features = []
    for computation in computations:
        if computation == "mean":
            computed_features.append(np.mean(feature_matrix, axis=1))
        elif computation == "min":
            computed_features.append(np.min(feature_matrix, axis=1))
        elif computation == "max":
            computed_features.append(np.max(feature_matrix, axis=1))
        elif computation == "std":
            computed_features.append(np.std(feature_matrix, axis=1))
        elif computation == "range":
            computed_features.append(np.ptp(feature_matrix, axis=1))
        elif computation == "mode":
            # Use scipy.stats.mode to get mode and count
            mode_result = mode(feature_matrix, axis=1)
            computed_features.append(mode_result.mode.flatten())
        elif computation == "median":
            computed_features.append(np.median(feature_matrix, axis=1))
        elif computation == "1st_quartile":
            computed_features.append(np.percentile(feature_matrix, 25, axis=1))
        elif computation == "3rd_quartile":
            computed_features.append(np.percentile(feature_matrix, 75, axis=1))
        # Add conditions for other computations

    return np.concatenate(computed_features)

In [ ]:
def extract_features(audio_path, features, global_computation):
    # Load the normalized and silence-removed audio
    audio, sr = librosa.load(audio_path)

    lst_spectral_flatness = []
    lst_spectral_centroid = []
    lst_mfcc = []
    lst_melspectrogram = []
    lst_chroma_stft = []
    lst_rms = []

    feature_list = []

    # Extract selected features
    for feature_name in features:
        if feature_name == "spectral_flatness":
            spectral_flatness = librosa.feature.spectral_flatness(y=audio)
            lst_spectral_flatness.append(global_feature_computation(spectral_flatness, global_computation))
        elif feature_name == "spectral_centroid":
            spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr = sr)
            lst_spectral_centroid.append(global_feature_computation(spectral_centroid, global_computation))
        elif feature_name == "mfcc":
            mfcc = librosa.feature.mfcc(y=audio, sr = sr)
            lst_mfcc.append(global_feature_computation(mfcc, global_computation))
        elif feature_name == "melspectrogram":
            mel_spectrum = librosa.feature.melspectrogram(y=audio, sr = sr)
            lst_melspectrogram.append(global_feature_computation(mel_spectrum, global_computation))
        elif feature_name == "chroma_stft":
            chroma_stft = librosa.feature.chroma_stft(y=audio, sr = sr)
            lst_chroma_stft.append(global_feature_computation(chroma_stft, global_computation))
        elif feature_name == "rms":
            rms = librosa.feature.rms(y=audio)
            lst_rms.append(global_feature_computation(rms, global_computation))


    feature_list.append(lst_spectral_flatness[0])
    feature_list.append(lst_spectral_centroid[0])
    feature_list.append(lst_mfcc[0])
    feature_list.append(lst_melspectrogram[0])
    feature_list.append(lst_chroma_stft[0])
    feature_list.append(lst_rms[0])


    # Combine the extracted features into a single feature vector
    feature_vector = np.concatenate(feature_list)

    return feature_vector


In [24]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')

    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")

## Model Training
- SVM

In [ ]:
features = ["spectral_flatness", "spectral_centroid", "mfcc", "chroma_stft", "melspectrogram", "rms"]
global_computation = ["mean"]
#global_computation = ["mean", "min", "max", "std", "range", "mode", "median"]

class_names = ["Neutral", "Cal", "Happiness", "Sadness", "Angry", "Fear", "Disgust", "Surprise"]

factor = 200

num_selected_features = 5

### Normalizing and removing the silence

In [ ]:
# Set the base directory where actor directories are located
base_directory = '/content/audio_speech_actors_01-24'
output_directory = '/content/processed_data'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# First loop: Normalize and remove silence
for actor_directory in os.listdir(base_directory):
    actor_path = os.path.join(base_directory, actor_directory)

    # Check if it's a directory
    if os.path.isdir(actor_path):

        # Iterate over audio files in the actor's directory
        for audio_file in os.listdir(actor_path):
            audio_path = os.path.join(actor_path, audio_file)

            # Normalize and remove silence
            normalise_remove_silence(audio_path, output_directory, factor)


In [ ]:
# List to store the paths of normalized audio files
normalized_audio_paths = []

for audio_file in os.listdir(output_directory):
    normalized_audio_path = os.path.join(output_directory, audio_file)

    # Append to the list of normalized audio paths
    normalized_audio_paths.append(normalized_audio_path)


In [ ]:
# Initialize lists for audio files, labels, and features
features_list = []
audio_files = []

for normalized_audio_path in normalized_audio_paths:
    extracted_features = extract_features(normalized_audio_path, features, global_computation)

    # Append to lists or perform further processing as needed
    audio_files.append(normalized_audio_path)
    features_list.append(extracted_features)


In [ ]:
# Convert features_list to a NumPy array if needed
features_list = np.array(features_list)

In [ ]:
features_list.shape

### labels
Here is the filename identifiers as per the official RAVDESS website:

- Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
- Vocal channel (01 = speech, 02 = song).
- Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
- Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
- Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
- Repetition (01 = 1st repetition, 02 = 2nd repetition).
- Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).


So, here's an example of an audio filename. 02-01-06-01-02-01-12.mp4 This means the meta data for the audio file is:

- Video-only (02)
- Speech (01)
- Fearful (06)
- Normal intensity (01)
- Statement "dogs" (02)
- 1st Repetition (01)
- 12th Actor (12) - Female (as the actor ID number is even)

In [ ]:
labels = []

for actor_directory in os.listdir(base_directory):
    actor_path = os.path.join(base_directory, actor_directory)

     #Check if it's a directory
    if os.path.isdir(actor_path):

        # Iterate over audio files in the actor's directory
        for audio_file in os.listdir(actor_path):
            # Split the file name using the "-" delimiter and take the third part as the emotion label
            parts = audio_file.split("-")
            emotion_label = parts[2]
            labels.append(emotion_label)

In [ ]:
labels = np.array(labels)
labels = labels.astype(np.int64)

In [ ]:
labels.shape

### Reading the csv of RAVDESS from kaggle

In [14]:
df = pd.read_csv('features.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,labels
0,-637.701233,104.299019,4.894947,20.494011,12.552954,2.851410,-6.633390,-4.091278,-10.423918,-6.406950,...,0.172893,-1.170210,-5.292450,-0.573319,1.019471,-3.492607,-3.468123,-1.214944,1.971239,male_calm
1,-596.908460,86.871936,9.470162,17.109819,11.198966,1.541056,-6.677264,-5.755428,-9.684472,-6.891256,...,0.033803,-1.986515,-5.103855,-1.253110,0.514896,-3.268317,-4.502895,0.167153,0.256732,male_calm
2,-698.086548,99.795929,1.892679,19.915264,7.532868,1.265761,-9.188656,-5.798194,-12.299710,-4.976400,...,-1.639542,-2.603761,-4.890347,-0.879222,-1.250208,-3.449960,-4.708529,-0.086224,-3.034044,male_calm
3,-279.141052,41.092949,-21.319229,7.802911,-13.140503,-9.407660,-15.580647,-6.097223,-24.700903,-9.640293,...,-10.305976,-4.092835,-5.817179,-10.731523,-0.823596,-15.885103,-2.014258,-6.173852,-5.331760,male_angry
4,-160.074686,17.576058,-2.147436,3.133417,-4.745002,-6.510771,-5.911591,-4.481506,-9.470598,-5.907823,...,-4.088007,-1.817639,-2.994765,-4.893176,-2.880436,-7.163400,-2.147345,-2.359248,-3.746217,male_angry


### Training the model

In [15]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(df.drop(['labels'], axis=1), df['labels'], test_size=0.2, random_state=42)

In [16]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((3456, 20), (3456,), (864, 20), (864,))

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# Initialize and train the SVM model
svc_model = SVC()
svc_model.fit(X_train, y_train)
#
#
# Initialize and train the KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
#
#
# Initialize and train the Gradient Boosting model
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
#
#
# Standardize the data before training neural networks
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
#
# Initialize and train the Neural Network model
nn_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
nn_model.fit(X_train, y_train)

MLPClassifier(max_iter=1000, random_state=42)

In [18]:
# Make predictions on the test set
y_pred_svc = svc_model.predict(X_test)
y_pred_knn = knn_model.predict(X_test)
y_pred_gb = gb_model.predict(X_test)
y_pred_nn = nn_model.predict(X_test)

In [19]:
y_pred_svc.shape, y_pred_knn.shape,  y_pred_gb.shape, y_pred_nn.shape

((864,), (864,), (864,), (864,))

In [20]:
# Accuracies
accuracy_svc = accuracy_score(y_test, y_pred_svc)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
accuracy_gb  = accuracy_score(y_test, y_pred_gb)
accuracy_nn  = accuracy_score(y_test, y_pred_nn)

In [21]:
models = []
accuracies = []

models.append(svc_model)
models.append(knn_model)
models.append(gb_model)
models.append(nn_model)

accuracies.append(accuracy_svc)
accuracies.append(accuracy_knn)
accuracies.append(accuracy_gb)
accuracies.append(accuracy_nn)

model_accuracy_df = pd.DataFrame(models, columns=['Models'])
model_accuracy_df['Accuracies'] = accuracies

In [22]:
model_accuracy_df

,Models,Accuracies
0,SVC(),0.237269
1,KNeighborsClassifier(),0.381944
2,([DecisionTreeRegressor(criterion='friedman_ms...,0.583333
3,"MLPClassifier(max_iter=1000, random_state=42)",0.495370


In [25]:
evaluate_model(svc_model, X_test, y_test)

Accuracy: 0.24
Precision: 0.21
Recall: 0.24


/Users/siakesophie/Local_Docs/ING_3/S9/Engineering_project/project/SERAPP/.env-serapp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
evaluate_model(knn_model, X_test, y_test)

In [ ]:
evaluate_model(gb_model, X_test, y_test)

In [ ]:
evaluate_model(nn_model, X_test, y_test)

## Training with DNN & CNN

In [ ]:
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import load_model, Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization

In [ ]:
Y = []
for i in range(labels.size):
  Y.append(labels[i])
  print(labels[i])

In [ ]:
# Reconstruction of the dataset with the lables from 01 to 07

display(features_list.shape)
df = pd.DataFrame(features_list)
df['labels'] = Y
df.to_csv('features.csv', index=False)
df.head()

In [ ]:
df_norm = df.drop("labels", axis=1)
df_norm.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df_norm)
print(scaler.mean_)
df_norm = scaler.transform(df_norm)

df_norm = pd.DataFrame(df_norm)
df_norm.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_norm, labels, test_size=0.2, random_state=42)

In [ ]:
# Let check the data shape
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
# For use the CNN model, let expand the data dimensions
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
# Modeling

model=Sequential()
model.add(Conv1D(128, kernel_size=5, strides = 1, padding='same', activation=tf.nn.relu, input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(Dropout(0.3))

model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation=tf.nn.relu))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(units=32, activation=tf.nn.relu))
model.add(Dropout(0.3))

model.add(Dense(units=9, activation=tf.nn.softmax))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
es = EarlyStopping(monitor  = "val_accuracy",
                   mode     = 'max',
                   verbose  = 0,
                   patience = 30)
mc = ModelCheckpoint('model_best.h5',
                     monitor        = 'val_accuracy',
                     mode           = 'max',
                     verbose        = 1,
                     save_best_only = True)

In [ ]:
model.summary()

In [ ]:
BATCH_SIZE = 100
EPOCHS = 30

history = model.fit(X_train, y_train, validation_split=0.2, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1, callbacks=[es, mc])

In [ ]:
print("Accuracy of our model on test data : " , model.evaluate(X_test,y_test)[1]*100 , "%")

epochs = [i for i in range(EPOCHS)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
test_acc = history.history['val_accuracy']
test_loss = history.history['val_loss']

fig.set_size_inches(20,6)
ax[0].plot(epochs , train_loss , label = 'Training Loss')
ax[0].plot(epochs , test_loss , label = 'Testing Loss')
ax[0].set_title('Training & Testing Loss')
ax[0].legend()
ax[0].set_xlabel("Epochs")

ax[1].plot(epochs , train_acc , label = 'Training Accuracy')
ax[1].plot(epochs , test_acc , label = 'Testing Accuracy')
ax[1].set_title('Training & Testing Accuracy')
ax[1].legend()
ax[1].set_xlabel("Epochs")
plt.show()